In [1]:
from data_reader import DataReader
import batch_for_bert
import datasets
import utils

In [2]:
dr = DataReader('for_bert')

dr.read_dataset(datasets.binary_classes)
dr.read_dataset(datasets.ternary_classes)



100%|██████████████████████████████████| 1000/1000 [00:00<00:00, 3675989.48it/s]


In [3]:
batch_size = 32

binary_dataset = dr.get_dataset('binary')
ternary_dataset = dr.get_dataset('ternary')

batched_binary = batch_for_bert.batch_dataset(binary_dataset, batch_size = 32)
batched_ternary = batch_for_bert.batch_dataset(ternary_dataset, batch_size = 32)

In [4]:
from bert_network import BertNetwork

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

simple_model = nn.Sequential(
                nn.Dropout(p = 0.5),
                nn.Linear(786, 2))
                

ce_loss = torch.nn.functional.cross_entropy

def my_optimizer_sgd(model_parameters):
    return optim.SGD(model_parameters, lr = 0.0001, momentum = 0.9)

In [6]:
network = BertNetwork(simple_model, ce_loss)
network.train(batched_binary['training tweets'], batched_binary['training tags'], 20, my_optimizer_sgd)

NameError: name 'SimpleNetwork' is not defined

In [ ]:
binary_predictions = network.predict(batched_binary['test tweets'])
utils.save_results(binary_predictions, 'results/binary_simple_network.txt')

In [ ]:
!perl graders/evaluate1.pl results/binary_simple_network.txt

In [ ]:
simple_model2 = nn.Sequential(
                nn.Dropout(p = 0.5),
                nn.Linear(786, 3))

network2 = BertNetwork(simple_model2, ce_loss)
network2.train(batched_ternary['training tweets'], batched_ternary['training tags'], 70, my_optimizer_sgd)

In [ ]:
ternary_predictions = network2.predict(batched_ternary['test tweets'])
utils.save_results(ternary_predictions, 'results/ternary_simple_network.txt')

In [ ]:
!perl graders/evaluate2.pl results/ternary_simple_network.txt